In [1]:
import pandas as pd
from pypfopt import EfficientFrontier
from pypfopt import risk_models
from pypfopt import expected_returns

import MySQLdb 
import numpy as np

In [2]:
ieo = pd.DataFrame()
price = pd.DataFrame()
grades = pd.DataFrame()


conn = MySQLdb.connect(host='tokenmetrics-restored-27-05.cxuzrhvtziar.us-east-1.rds.amazonaws.com', user='admin', passwd='WiG8Rled2cTvZ5JibJui',db='tokenmetrics')
cursor = conn.cursor()

query = '''SELECT * FROM icos;'''
data = pd.read_sql_query(query, conn)
ieo = pd.concat([ieo,data])

query = 'SELECT * FROM ico_price_daily_summaries where currency ="USD";'
data = pd.read_sql_query(query, conn)
price = pd.concat([price,data])

query = 'SELECT * FROM ico_ml_grade_history;'
data = pd.read_sql_query(query, conn)
grades = pd.concat([grades,data])
    
conn.close() 

In [3]:
ieo.sort_values('ico_market_cap', ascending=False, inplace=True)
ieo = ieo.drop(ieo[ieo['id'] == 3245].index)
ieo = ieo[:150]
ieo['weight'] = ieo['ico_market_cap'] / ieo['ico_market_cap'].sum()
ieo = pd.merge(ieo,price[['ico_id','ico_symbol']], left_on='id',right_on='ico_id',how='inner')
ieo = ieo.drop_duplicates('id')
weight = ieo[['ico_symbol','weight']]
weight = weight.sort_values('ico_symbol')

ieos = ieo['id'][:150]
df = price[price['ico_id'].isin(ieos)]
df = df.drop_duplicates(subset=['ico_symbol','date']).sort_values(by='date')


#temp = temp.dropna()

In [4]:
temp = df['close'].groupby([df['date'],df['ico_symbol']]).mean()
temp = temp.unstack().sort_index()
temp.index = pd.to_datetime(temp.index)

In [5]:
# Calculate expected returns and sample covariance
mu = expected_returns.mean_historical_return(temp.loc['2019-01-01':])
S = risk_models.sample_cov(temp.loc['2019-01-01':])

# Optimise for maximal Sharpe ratio
ef = EfficientFrontier(mu, S)
raw_weights = ef.max_sharpe()
cleaned_weights = ef.clean_weights()
ef.save_weights_to_file("weights.csv")  # saves to file
print(cleaned_weights)
ef.portfolio_performance(verbose=True)

OrderedDict([('ABBC', 0.0), ('ADA', 0.0), ('ALGO', 0.0), ('AMPL', 0.0), ('AR', 0.1147), ('ATOM', 0.0), ('BAT', 0.0), ('BAY', 0.01456), ('BCD', 0.0), ('BCH', 0.0), ('BCN', 0.0), ('BNB', 0.0), ('BNT', 0.0), ('BOTX', 0.01868), ('BSV', 0.0), ('BTC', 0.0), ('BTG', 0.0), ('BTM', 0.0), ('BTRS', 0.00406), ('BTT', 0.0), ('BZT', 0.0), ('CCC', 0.01699), ('CCXX', 0.0), ('CEL', 0.09814), ('CELO', 0.0), ('CENNZ', 0.0), ('CKB', 0.15005), ('COMP', 0.01626), ('CRO', 0.0), ('DAI', 0.0), ('DASH', 0.0), ('DCR', 0.0), ('DEV', 0.0), ('DGB', 0.0), ('DOGE', 0.05055), ('EKT', 0.0), ('ELA', 0.0), ('ELAMA', 0.13674), ('ENJ', 0.0), ('EON', 0.0), ('EOS', 0.0), ('ETC', 0.0), ('ETH', 0.0), ('FST', 0.00193), ('FXC', 0.0), ('GAP', 0.0), ('GTN', 0.02151), ('GXC', 0.0), ('HBAR', 0.0), ('HC', 0.0), ('HDAC', 0.04564), ('HEDG', 0.0), ('HIVE', 0.0), ('HMR', 0.0), ('HOT', 0.0), ('HT', 0.0), ('ICX', 0.0), ('INB', 0.0), ('INO', 0.0), ('IOST', 0.0), ('KBC', 0.0), ('KCS', 0.0), ('KIN', 0.0), ('KMD', 0.0), ('KNC', 0.0), ('LBC', 0

C:\Users\Aagam\Anaconda3\lib\site-packages\pypfopt\risk_models.py:68: UserWarning: The covariance matrix is non positive semidefinite. Amending eigenvalues.
  "The covariance matrix is non positive semidefinite. Amending eigenvalues."
C:\Users\Aagam\Anaconda3\lib\site-packages\pypfopt\risk_models.py:87: UserWarning: Could not fix matrix. Please try a different risk model.
  warnings.warn("Could not fix matrix. Please try a different risk model.")


(4.644114114535931, 0.4123697994566162, 11.213513018240358)

In [6]:
weights = pd.DataFrame.from_dict(dict(cleaned_weights), orient='index')
weights = weights[weights[0] > 0]

price = price.sort_values('date',ascending=False)
price = price[price['date'] == price['date'].iloc[0]]

allocation = pd.DataFrame.from_dict(allocation,orient='index')
allocation.columns = ['Token Volume']
allocation['ico_symbol'] = allocation.index

allocation = pd.merge(allocation, price[['close','ico_symbol']])

allocation['Holding Amount'] = abs(allocation['Token Volume']) * allocation['close']
allocation['Position'] = np.where(allocation['Token Volume']<0, 'Short', 'Long')
allocation

In [7]:
allocation = pd.DataFrame()
allocation['ico_symbol'] = weights.index.tolist()
allocation['weights'] = weights[0].tolist()

In [8]:
close = []
for symbol in allocation['ico_symbol'].tolist():
    temp = price[price['ico_symbol'] == symbol]
    temp = temp.sort_values('date',ascending=False)
    temp = temp[temp['date'] == temp['date'].iloc[0]]
    close.append(temp['close'].values[0])

In [9]:
allocation['price'] = close

In [10]:
allocation['Holding Amount'] = allocation['weights'] * 10000

In [11]:
allocation['% Holding'] = allocation['Holding Amount'] / allocation['Holding Amount'].sum() * 100

In [12]:
allocation.sort_values('% Holding', ascending=False)

,ico_symbol,weights,price,Holding Amount,% Holding
6,CKB,0.15005,0.005622,1500.5,15.004700
9,ELAMA,0.13674,0.110921,1367.4,13.673727
0,AR,0.11470,1.561404,1147.0,11.469771
19,UMA,0.10144,3.827458,1014.4,10.143797
5,CEL,0.09814,0.404295,981.4,9.813804
22,YFI,0.08226,4230.514648,822.6,8.225835
8,DOGE,0.05055,0.003174,505.5,5.054899
14,ONE,0.04665,0.008803,466.5,4.664907
12,HDAC,0.04564,0.029780,456.4,4.563909
17,SNX,0.04370,3.396879,437.0,4.369913


### Views

In [13]:
from pypfopt import black_litterman
from pypfopt.black_litterman import BlackLittermanModel
from pypfopt.efficient_frontier import EfficientFrontier

grades = grades[grades['ico_id'].isin(ieos)]
grades = pd.merge(grades,price[['ico_id','ico_symbol']], on='ico_id',how='inner')
grades = grades.drop_duplicates(['ico_id','date'])
grades = grades.sort_values('date', ascending=False)
grades = grades[grades['date'] == grades['date'].iloc[0]]

In [14]:
grades['view'] = grades['final_grade'] / grades['final_grade'].sum()
views = grades[['ico_symbol','view']]
views.set_index('ico_symbol', inplace=True)
views = views.to_dict()
viewdict = views['view']

temp = df['close'].groupby([df['date'],df['ico_symbol']]).mean()
temp = temp.unstack().sort_index()
temp.index = pd.to_datetime(temp.index)

cov_matrix = temp.loc['2019-01-01':].cov()

In [15]:
bl = BlackLittermanModel(cov_matrix, pi="equal", absolute_views=viewdict, omega='default')

rets = bl.bl_returns()
ef = EfficientFrontier(rets, cov_matrix, weight_bounds=(0, 0.1))

# OR use return-implied weights
delta = black_litterman.market_implied_risk_aversion(temp.loc['2019-01-01':])
bl.bl_weights(delta.values)
weights = bl.clean_weights()

C:\Users\Aagam\Anaconda3\lib\site-packages\pypfopt\black_litterman.py:442: RuntimeWarning: invalid value encountered in multiply
  A = risk_aversion * self.cov_matrix
C:\Users\Aagam\Anaconda3\lib\site-packages\pypfopt\base_optimizer.py:87: RuntimeWarning: invalid value encountered in less
  clean_weights[np.abs(clean_weights) < cutoff] = 0


In [16]:
weights = pd.DataFrame.from_dict(dict(weights), orient='index')
weights = weights[weights[0] > 0]

In [17]:
weights

,0


In [12]:
da = DiscreteAllocation(weights, latest_prices, total_portfolio_value=10000,short_ratio=0.0001)
allocation, leftover = da.lp_portfolio()
print("Discrete allocation:", allocation)
print("Funds remaining: ${:.2f}".format(leftover))

Discrete allocation: {'HT': 5.0, 'TRX': 555171.0, 'XLM': 1106.0, 'XTZ': 28.0, 'LBC': -5.0, 'XRP': -4.0}
Funds remaining: $0.08


In [13]:
price = price.sort_values('date',ascending=False)
price = price[price['date'] == price['date'].iloc[0]]

allocation = pd.DataFrame.from_dict(allocation,orient='index')
allocation.columns = ['Token Volume']
allocation['ico_symbol'] = allocation.index

allocation = pd.merge(allocation, price[['close','ico_symbol']])

allocation['Holding Amount'] = abs(allocation['Token Volume']) * allocation['close']
allocation['Position'] = np.where(allocation['Token Volume']<0, 'Short', 'Long')
allocation

,Token Volume,ico_symbol,close,Holding Amount,Position
0,5.0,HT,4.238911,21.194553,Long
1,555171.0,TRX,0.017635,9790.570328,Long
2,1106.0,XLM,0.096894,107.164875,Long
3,28.0,XTZ,2.895190,81.065320,Long
4,-5.0,LBC,0.025488,0.127438,Short
5,-4.0,XRP,0.199965,0.799861,Short
